# ESG

---

## http://esgcompare.org/

https://www.kaggle.com/datasets/alistairking/public-company-esg-ratings-dataset

Nasdaq_api_key = PYD3bWYxsAJ8ZP3YQuqz

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
#import time
import scipy.stats as st
from scipy.stats import sem
from scipy.stats import linregress
import json


# Import any API key
#from api_keys import weather_api_key


# Study data files
esg_data_path = "data.csv"


# Reading the mouse data and the study results
esg_data = pd.read_csv(esg_data_path)


# Displaying the data table for preview
esg_data.head()

,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,...,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/ef50b4a2b263c84...,https://thewaltdisneycompany.com/,A,High,BB,...,BB,Medium,510,316,321,1147,19-04-2022,BBB,High,1744489
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,https://static.finnhub.io/logo/9253db78-80c9-1...,https://www.gm.com/,A,High,BB,...,B,Medium,510,303,255,1068,17-04-2022,BBB,High,1467858
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,https://static.finnhub.io/logo/f153dcda-80eb-1...,https://www.grainger.com/,B,Medium,BB,...,B,Medium,255,385,240,880,19-04-2022,BB,Medium,277135
3,mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,https://static.finnhub.io/logo/26868a62-80ec-1...,https://mohawkind.com/,A,High,B,...,BB,Medium,570,298,303,1171,18-04-2022,BBB,High,851968
4,lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/1cd144d2-80ec-1...,https://www.livenationentertainment.com/,BBB,High,BB,...,B,Medium,492,310,250,1052,18-04-2022,BBB,High,1335258


In [2]:
top_companies = esg_data.sort_values("total_score", ascending=False)
top_companies.head(10)

,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,...,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik
452,cop,Conocophillips,USD,"NEW YORK STOCK EXCHANGE, INC.",Energy,https://static.finnhub.io/logo/aacfcb5e-80eb-1...,http://www.conocophillips.com/,AA,Excellent,A,...,BB,Medium,687,544,305,1536,16-04-2022,A,High,1163165
176,msft,Microsoft Corp,USD,NASDAQ NMS - GLOBAL MARKET,Technology,https://static.finnhub.io/logo/25e313016e9fac0...,https://www.microsoft.com/en-us,AA,Excellent,BBB,...,BB,Medium,715,443,375,1533,27-04-2022,A,High,789019
302,eix,Edison International,USD,"NEW YORK STOCK EXCHANGE, INC.",Utilities,https://static.finnhub.io/logo/cbccd574-80eb-1...,https://www.edison.com/,AA,Excellent,A,...,BB,Medium,700,513,315,1528,17-04-2022,A,High,827052
18,abnb,Airbnb Inc,USD,NASDAQ NMS - GLOBAL MARKET,Hotels Restaurants and Leisure,https://static2.finnhub.io/file/publicdatany/f...,https://www.airbnb.co.in/,A,High,A,...,BBB,High,505,570,400,1475,10-06-2022,A,High,1559720
250,apd,Air Products and Chemicals Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Chemicals,https://static.finnhub.io/logo/74581b6e-80eb-1...,https://www.airproducts.com/,AA,Excellent,BBB,...,BB,Medium,700,442,300,1442,16-04-2022,A,High,2969
137,amzn,Amazon.com Inc,USD,NASDAQ NMS - GLOBAL MARKET,Retail,https://static.finnhub.io/logo/967bf7b0-80df-1...,https://www.amazon.com/,AA,Excellent,BB,...,BBB,High,668,305,460,1433,16-04-2022,A,High,1018724
487,nem,Newmont Corporation,USD,"NEW YORK STOCK EXCHANGE, INC.",Metals and Mining,https://static.finnhub.io/logo/39e32600-80ec-1...,https://newmont.com/,AA,Excellent,BB,...,BB,Medium,675,389,355,1419,18-04-2022,A,High,1164727
195,pep,PepsiCo Inc,USD,NASDAQ NMS - GLOBAL MARKET,Beverages,https://static2.finnhub.io/file/publicdatany/f...,https://www.pepsico.com/,AA,Excellent,BB,...,BB,Medium,719,340,358,1417,18-04-2022,A,High,77476
494,exc,Exelon Corp,USD,NASDAQ NMS - GLOBAL MARKET,Utilities,https://static.finnhub.io/logo/d63a05d0-80eb-1...,https://www.exeloncorp.com/,AA,Excellent,BBB,...,BB,Medium,694,403,316,1413,17-04-2022,A,High,1109357
372,pxd,Pioneer Natural Resources Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Energy,https://static.finnhub.io/logo/5d0d741c-80ec-1...,https://www.pxd.com/,AA,Excellent,BBB,...,BB,Medium,700,412,300,1412,18-04-2022,A,High,1038357


In [3]:
bottom_companies = esg_data.sort_values("total_score", ascending=True)
bottom_companies.head(10)

,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,...,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik
655,pola,Polar Power Inc,USD,NASDAQ NMS - GLOBAL MARKET,Electrical Equipment,https://static.finnhub.io/logo/09caf854-81d9-1...,https://polarpower.com/,B,Medium,B,...,B,Medium,200,200,200,600,05-06-2022,B,Medium,1622345
704,gva,Granite Construction Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Construction,https://static.finnhub.io/logo/f0d5c3ba-80eb-1...,https://www.graniteconstruction.com/,B,Medium,B,...,B,Medium,200,200,200,600,07-11-2022,B,Medium,861459
228,zion,Zions Bancorporation NA,USD,NASDAQ NMS - GLOBAL MARKET,Banking,https://static.finnhub.io/logo/1ab22e6c-80e1-1...,https://www.zionsbank.com/,B,Medium,B,...,B,Medium,200,200,200,600,20-04-2022,B,Medium,109380
76,afaq,AF Acquisition Corp,USD,NASDAQ NMS - GLOBAL MARKET,NaN,NaN,NaN,B,Medium,B,...,B,Medium,200,200,200,600,21-06-2022,B,Medium,1841661
303,glw,Corning Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Electrical Equipment,https://static.finnhub.io/logo/ec2c31c6-80eb-1...,https://www.corning.com/,B,Medium,B,...,B,Medium,200,200,200,600,16-04-2022,B,Medium,24741
395,tel,TE Connectivity Ltd,USD,"NEW YORK STOCK EXCHANGE, INC.",Electrical Equipment,https://static.finnhub.io/logo/81c63b2c-80ec-1...,https://www.te.com/,B,Medium,B,...,B,Medium,200,200,200,600,19-04-2022,B,Medium,1385157
396,tfx,Teleflex Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Health Care,https://static4.finnhub.io/file/publicdatany2/...,https://www.teleflex.com/,B,Medium,B,...,B,Medium,200,200,200,600,19-04-2022,B,Medium,96943
595,xl,XL Fleet Corp,USD,"NEW YORK STOCK EXCHANGE, INC.",Auto Components,https://static.finnhub.io/logo/c558526758aff6a...,https://xlfleet.com/,B,Medium,B,...,B,Medium,200,200,200,600,22-04-2022,B,Medium,1772720
72,aese,Allied Esports Entertainment Inc,USD,NASDAQ NMS - GLOBAL MARKET,Hotels Restaurants and Leisure,https://static.finnhub.io/logo/60068062-80db-1...,https://ir.alliedesportsent.com/,B,Medium,B,...,B,Medium,200,200,200,600,10-06-2022,B,Medium,1708341
313,ko,Coca-Cola Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Beverages,https://static.finnhub.io/logo/12f0377a-80ec-1...,https://www.coca-colacompany.com/,B,Medium,B,...,B,Medium,200,201,200,601,16-04-2022,B,Medium,21344


In [4]:
esg_data.count()


ticker                  722
name                    722
currency                722
exchange                722
industry                709
logo                    702
weburl                  706
environment_grade       722
environment_level       722
social_grade            722
social_level            722
governance_grade        722
governance_level        722
environment_score       722
social_score            722
governance_score        722
total_score             722
last_processing_date    722
total_grade             722
total_level             722
cik                     722
dtype: int64

In [5]:
esg_data.nunique()

ticker                  722
name                    722
currency                  5
exchange                  2
industry                 47
logo                    702
weburl                  706
environment_grade         5
environment_level         3
social_grade              6
social_level              4
governance_grade          4
governance_level          3
environment_score       197
social_score            186
governance_score         80
total_score             441
last_processing_date     30
total_grade               4
total_level               2
cik                     722
dtype: int64

In [6]:

esg_industry_agg = esg_data.groupby(["industry"])["total_score"].agg(("mean", "median", "var", "std", "sem"))
esg_industry_counts = esg_data.groupby(["industry"])["total_score"].value_counts()
esg_industry_agg["# of companies"] = esg_data.groupby(["industry"])["total_score"].count()
esg_industry_agg

,mean,median,var,std,sem,# of companies
industry,,,,,,
Aerospace & Defense,633.000000,633.0,NaN,NaN,NaN,1
Aerospace and Defense,1089.700000,1136.0,24886.011111,157.753007,49.885881,10
Airlines,838.714286,746.0,36032.904762,189.823351,71.746483,7
Auto Components,833.333333,696.0,105349.333333,324.575620,187.393822,3
Automobiles,859.250000,873.0,42856.250000,207.017511,103.508756,4
Banking,898.310345,886.0,33559.507389,183.192542,34.018001,29
Beverages,1061.000000,1113.0,115992.000000,340.575983,170.287991,4
Biotechnology,866.910714,822.5,31003.610065,176.078420,23.529469,56
Building,1074.200000,1123.5,40144.400000,200.360675,63.359609,10


In [7]:
esg_stats_sorted = esg_industry_agg.sort_values("mean", ascending=False)
esg_stats_sorted



,mean,median,var,std,sem,# of companies
industry,,,,,,
Utilities,1190.633333,1199.5,35381.067816,188.098559,34.341941,30
Tobacco,1165.000000,1165.0,18.000000,4.242641,3.000000,2
Industrial Conglomerates,1157.000000,1141.0,1216.000000,34.871192,20.132892,3
Packaging,1156.000000,1157.0,2290.000000,47.853944,23.926972,4
Energy,1142.263158,1151.0,46887.204678,216.534535,49.676429,19
Food Products,1131.071429,1127.0,9614.994505,98.056078,26.206589,14
Distributors,1109.666667,1120.0,1640.333333,40.501029,23.383280,3
Aerospace and Defense,1089.700000,1136.0,24886.011111,157.753007,49.885881,10
Construction,1082.666667,1187.5,61952.666667,248.902926,101.614194,6


In [8]:
esg_stats_refined = esg_stats_sorted[(esg_stats_sorted["# of companies"]>=10)]
esg_stats_refined.head(10)



,mean,median,var,std,sem,# of companies
industry,,,,,,
Utilities,1190.633333,1199.5,35381.067816,188.098559,34.341941,30
Energy,1142.263158,1151.0,46887.204678,216.534535,49.676429,19
Food Products,1131.071429,1127.0,9614.994505,98.056078,26.206589,14
Aerospace and Defense,1089.700000,1136.0,24886.011111,157.753007,49.885881,10
Building,1074.200000,1123.5,40144.400000,200.360675,63.359609,10
Chemicals,1058.894737,1132.0,55246.654971,235.046070,53.923267,19
Machinery,1048.047619,1125.0,29579.647619,171.987347,37.530716,21
Technology,1031.317460,1103.0,38448.639529,196.083246,24.704167,63
Semiconductors,1030.583333,1100.0,27034.862319,164.422816,33.562667,24


In [9]:
avg_ESG_score_industry = esg_stats_refined["mean"]

avg_esg = Top_avg_ESG_score_industry.plot(kind="bar", facecolor="blue", width=.75,
                                                  title="Top Average ESG Score by Industry",
                                                  figsize=(12,6),
                                                  xlabel="Industry",
                                                  ylabel="Average ESG Score")

NameError: name 'Top_avg_ESG_score_industry' is not defined

In [ ]:
bottom_esg_stats = esg_stats_refined.sort_values("mean", ascending=True)

bottom_esg_stats.head(10)


In [ ]:
industry_counts = esg_data.groupby(["exchange"])["total_score"].count()
industry_counts

In [ ]:
esg_enviro_grades = esg_data.groupby(["environment_grade"])["ticker"].count()
esg_enviro_grades_pie = esg_enviro_grades.plot(kind="pie",
                                                  title="Top Average ESG Score by Industry",
                                                  figsize=(12,6),
                                                  )

In [ ]:
esg_stats_util = esg_data.loc[esg_data['industry'] == 'Utilities',:]
esg_stats_util = esg_stats_util.sort_values('total_score',ascending = False)
esg_stats_util

In [ ]:
esg_util_series_env = esg_data[(esg_data['industry'] == 'Utilities')]["environment_score"]
esg_util_series_soc = esg_data[(esg_data['industry'] == 'Utilities')]["social_score"]
esg_util_series_gov = esg_data[(esg_data['industry'] == 'Utilities')]["governance_score"]
esg_util_series_tot = esg_data[(esg_data['industry'] == 'Utilities')]["total_score"]

In [ ]:
data = [esg_util_series_env, esg_util_series_soc, esg_util_series_gov, esg_util_series_tot]

# Create a boxplot for the data
plt.boxplot(data)

# Add labels to the x-axis
plt.xticks([1,2,3,4], ['Environmental','Social','Governance','Total'])

# Add a title and labels
plt.title('Utilities Industry ESG Scores')
plt.xlabel('Score Type')
plt.ylabel('Score')

# Show the plot
plt.show()

In [ ]:
esg_util_series_tot = esg_data[(esg_data['industry'] == 'Utilities')]["total_score"]
esg_ic_series_tot = esg_data[(esg_data['industry'] == 'Industrial Conglomerates')]["total_score"]
esg_pack_series_tot = esg_data[(esg_data['industry'] == 'Packaging')]["total_score"]
esg_enrg_series_tot = esg_data[(esg_data['industry'] == 'Energy')]["total_score"]
esg_food_series_tot = esg_data[(esg_data['industry'] == 'Food Products')]["total_score"]




In [ ]:
data = [esg_util_series_tot, esg_ic_series_tot , esg_pack_series_tot, esg_enrg_series_tot, esg_food_series_tot ]

# Create a boxplot for the data
plt.boxplot(data)

# Add labels to the x-axis
plt.xticks([1,2,3,4,5], ['Utilities','Industrial Conglomerates','Packaging','Energy','Food Products'], rotation=45)

# Add a title and labels
plt.title('Total ESG Scores, Top 5 Industries')
plt.xlabel('Score Type')
plt.ylabel('Score')

# Show the plot
plt.show()

In [ ]:
# Define a function to create Linear Regression plots
def linear_regression(x_values, y_values):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    plt.annotate(line_eq,(5.8,0.8),fontsize=15,color="red")
    correlation = st.pearsonr(x_values,y_values)
    print(f"The correlation is: {round(correlation[0],2)}")
    print(f"The r-squared is: {rvalue**2}")
    plt.show()

In [ ]:
# Linear regression on Northern Hemisphere
plt.title("Title")
plt.xlabel("Governance Score")
plt.ylabel("Total Score")
linear_regression(esg_data["governance_score"], esg_data["total_score"])

In [ ]:
plt.title("Title")
plt.xlabel("Social Score")
plt.ylabel("Total Score")
linear_regression(esg_data["social_score"], esg_data["total_score"])

In [ ]:
plt.title("Title")
plt.xlabel("Environment Score")
plt.ylabel("Total Score")
linear_regression(esg_data["environment_score"], esg_data["total_score"])

In [ ]:
https://esgapiservice.com/api/Scores

b2c6f24b34915131bd01427cfe9fab8a

In [ ]:
stock_data = []

esg_stock_list = ['cop', 'msft', 'eix']

for stock in esg_stock_list:
    
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={stock}&interval=60min&month=2019-1&outputsize=full&apikey=7CJJK2F0B3CJS22J"
    print(url)
    try:
        stock_info_return = requests.get(url)
        data = stock_info_return.json()
        five_yr_start = data['Monthly Time Series']
        #['2019-01-31']['4. close']
        print(five_yr_start)
        #five_yr_end = also_data['Monthly Time Series']['2024-01-31']['4. close']


        #stock_data.append({"Ticker": stock,
                           #"5 YR Open": five_yr_start, 
                           #"5 YR Close": five_yr_end})
    except:
        print("skipping")

In [ ]:
stock = "ibm"
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={stock}&interval=60min&month=2019-1&outputsize=full&apikey=OZ2ZL66PFIZ1B6GT"
url

In [11]:

ticker = "cop"
url = "https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=cop&interval=60min&month=2019-1&outputsize=full&apikey=7CJJK2F0B3CJS22J"
stock_info_return = requests.get(url)
stock_info_return.json()



{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}

In [ ]:
also_data = stock_info_return.json()
five_yr_start = also_data['Monthly Time Series']['2019-01-31']['4. close']
print(five_yr_start)

In [ ]:
five_yr_end = also_data['Monthly Time Series']['2024-01-31']['4. close']
print(five_yr_end)

In [ ]:
perc_change=(((float(five_yr_end))-(float(five_yr_start)))/(float(five_yr_start)))*100

perc_change

In [ ]:
close = stock_info_return['Meta Data']['Monthly Time Series']
print(close)

In [ ]:
Stock_info_2 = requests.get('https://alphavantageapi.co/timeseries/analytics?SYMBOLS=AAPL,MSFT,IBM&RANGE=2023-07-01&RANGE=2023-08-31&INTERVAL=DAILY&OHLC=close&CALCULATIONS=MEAN,STDDEV,CORRELATION&apikey=OZ2ZL66PFIZ1B6GT')
Stock_info_2.json()

In [ ]:
data = Stock_info_2.json()

something = data["payload"]['RETURNS_CALCULATIONS']['MEAN']
print(something)